In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("stores_sales_forecasting.csv",encoding="latin1")

In [3]:
df.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         2121 non-null   int64  
 1   Order ID       2121 non-null   object 
 2   Order Date     2121 non-null   object 
 3   Ship Date      2121 non-null   object 
 4   Ship Mode      2121 non-null   object 
 5   Customer ID    2121 non-null   object 
 6   Customer Name  2121 non-null   object 
 7   Segment        2121 non-null   object 
 8   Country        2121 non-null   object 
 9   City           2121 non-null   object 
 10  State          2121 non-null   object 
 11  Postal Code    2121 non-null   int64  
 12  Region         2121 non-null   object 
 13  Product ID     2121 non-null   object 
 14  Category       2121 non-null   object 
 15  Sub-Category   2121 non-null   object 
 16  Product Name   2121 non-null   object 
 17  Sales          2121 non-null   float64
 18  Quantity

In [6]:
df = df[
    [
        "Order Date",
        "Region",
        "Sales",
        "Profit",
        "Quantity"
    ]
]

df.head()


,Order Date,Region,Sales,Profit,Quantity
0,11/8/2016,South,261.9600,41.9136,2
1,11/8/2016,South,731.9400,219.5820,3
2,10/11/2015,South,957.5775,-383.0310,5
3,6/9/2014,West,48.8600,14.1694,7
4,6/9/2014,West,1706.1840,85.3092,9


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Order Date  2121 non-null   object 
 1   Region      2121 non-null   object 
 2   Sales       2121 non-null   float64
 3   Profit      2121 non-null   float64
 4   Quantity    2121 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 83.0+ KB


In [10]:
df.loc[:, "Order Date"] = pd.to_datetime(df["Order Date"])

In [16]:
df.isna().sum()

,0
Order Date,0
Region,0
Sales,0
Profit,0
Quantity,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Order Date  2121 non-null   datetime64[ns]
 1   Region      2121 non-null   object        
 2   Sales       2121 non-null   float64       
 3   Profit      2121 non-null   float64       
 4   Quantity    2121 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 83.0+ KB


In [13]:
monthly_sales = (
    df
    .groupby(
        [
            pd.Grouper(key="Order Date", freq="ME"),
            "Region"
        ]
    )
    .agg(
        total_sales=("Sales", "sum"),
        total_profit=("Profit", "sum"),
        total_quantity=("Quantity", "sum")
    )
    .reset_index()
)

monthly_sales.head()

,Order Date,Region,total_sales,total_profit,total_quantity
0,2014-01-31,Central,506.358,20.9235,11
1,2014-01-31,East,199.004,-21.6058,12
2,2014-01-31,South,4378.728,1130.2568,32
3,2014-01-31,West,1158.435,-324.1080,15
4,2014-02-28,Central,439.310,64.0797,13


In [14]:
# Regions present
print(monthly_sales["Region"].value_counts())

# Date range present
print(monthly_sales["Order Date"].min(), monthly_sales["Order Date"].max())

Region
Central    48
South      48
West       48
East       47
Name: count, dtype: int64
2014-01-31 00:00:00 2017-12-31 00:00:00


In [15]:
monthly_sales.to_csv(
    "data_processed_monthly_sales_by_region.csv",
    index=False
)